In [0]:
# input_image = np.expand_dims(numpy_image, axis=0)

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

!ls

drive  sample_data


In [0]:
%matplotlib inline

import matplotlib
matplotlib.use("Agg")
import pandas as pd
from keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from random import shuffle
import os

In [0]:
 df = pd.read_csv('/content/drive/My Drive/mini-proj/info.csv')

In [0]:
def csv_image_generator(imgpath, bs, lb, mode="train", aug=None):
    # open the CSV file for reading
    
    imgs = [x for x in os.listdir(imgpath) if x.split('.')[-1] == 'png']
    print(len(imgs))
    shuffle(imgs)
    cnt = 0
    while True:
        labels = []
        images = []
        while len(images) < bs:
            fn = imgs[cnt]
            oimg = image.load_img(os.path.join(imgpath,fn), target_size=(224,224))
            oimg = image.img_to_array(oimg)
            num = int(fn.split('.')[0].split('-')[1])
            cnt = (cnt+1)%len(imgs)
            if cnt == 0 and mode == 'eval':
                break

            label = int(df.loc[num-1]["'src-data'"])
            images.append(oimg)
            labels.append(label)

        labels = lb.transform(np.array(labels))

        if aug is not None:
            (images, labels) = next(aug.flow(np.array(images),
                    labels, batch_size=bs))

        yield (np.array(images), labels)

In [0]:

NUM_EPOCHS = 30
BS = 25

# initialize the total number of training and testing image
NUM_TRAIN_IMAGES = 1042

TRAIN_PATH = '/content/drive/My Drive/mini-proj/imgs/train'
TEST_PATH = '/content/drive/My Drive/mini-proj/imgs/test'
labels = [1, 2]
testLabels = []
imgs = os.listdir(TEST_PATH)
for fn in imgs:
    num = int(fn.split('.')[0].split('-')[1])
    testLabels.append(int(df.loc[num-1]["'src-data'"]))

In [0]:
NUM_TEST_IMAGES = len(testLabels)
NUM_EPOCHS = 30

In [0]:

# create the label binarizer for one-hot encoding labels, then encode
# the testing labels
lb = LabelBinarizer()
lb.fit(labels)
testLabels = lb.transform(testLabels)

# construct the training image generator for data augmentation
aug = image.ImageDataGenerator(rotation_range=0, zoom_range=0.1,
    width_shift_range=0, height_shift_range=0, shear_range=0.1,
    horizontal_flip=True, fill_mode="nearest")

# initialize both the training and testing image generators
trainGen = csv_image_generator(TRAIN_PATH, BS, lb,
    mode="train", aug=aug)
testGen = csv_image_generator(TEST_PATH, BS, lb,
    mode="train", aug=None)

# initialize our Keras model and compile it
# model = MiniVGGNet.build(64, 64, 3, len(lb.classes_))
# opt = SGD(lr=1e-2, momentum=0.9, decay=1e-2 / NUM_EPOCHS)
# model.compile(loss="categorical_crossentropy", optimizer=opt,
#     metrics=["accuracy"])

In [0]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.
87916544/87910968 [==============================] - 3s 0us/step


In [0]:
# model.fit_generator(...)

# train the network
print("[INFO] training w/ generator...")
H = model.fit_generator(
    trainGen,
    steps_per_epoch=NUM_TRAIN_IMAGES // BS,
    validation_data=testGen,
    validation_steps=NUM_TEST_IMAGES // BS,
    epochs=NUM_EPOCHS)

[INFO] training w/ generator...
Instructions for updating:
Use tf.cast instead.
Epoch 1/30
41/41 [==============================] - 549s 13s/step - loss: 3.6814 - val_loss: 3.5460
Epoch 2/30
41/41 [==============================] - 26s 624ms/step - loss: 3.7897 - val_loss: 3.5460
Epoch 3/30
41/41 [==============================] - 20s 496ms/step - loss: 3.8054 - val_loss: 3.5460
Epoch 4/30
41/41 [==============================] - 19s 458ms/step - loss: 3.6954 - val_loss: 3.5460
Epoch 5/30
41/41 [==============================] - 19s 458ms/step - loss: 3.7583 - val_loss: 3.5460
Epoch 6/30
41/41 [==============================] - 19s 460ms/step - loss: 3.7583 - val_loss: 3.5460
Epoch 7/30
41/41 [==============================] - 20s 493ms/step - loss: 3.7897 - val_loss: 3.5460
Epoch 8/30
41/41 [==============================] - 19s 456ms/step - loss: 3.7740 - val_loss: 3.5460
Epoch 9/30
41/41 [==============================] - 19s 459ms/step - loss: 3.7740 - val_loss: 3.5460
Epoch 10/30


In [0]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='sparse_categorical_crossentropy')

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

In [0]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
# model.fit_generator(...)

H = model.fit_generator(
    trainGen,
    steps_per_epoch=NUM_TRAIN_IMAGES // BS,
    validation_data=testGen,
    validation_steps=NUM_TEST_IMAGES // BS,
    epochs=NUM_EPOCHS)

Epoch 1/30
41/41 [==============================] - 23s 570ms/step - loss: 3.7425 - val_loss: 3.5460
Epoch 2/30
41/41 [==============================] - 19s 472ms/step - loss: 3.7583 - val_loss: 3.5460
Epoch 3/30
41/41 [==============================] - 19s 464ms/step - loss: 3.7897 - val_loss: 3.5460
Epoch 4/30
41/41 [==============================] - 19s 463ms/step - loss: 3.7897 - val_loss: 3.5460
Epoch 5/30
41/41 [==============================] - 19s 464ms/step - loss: 3.7583 - val_loss: 3.5460
Epoch 6/30
41/41 [==============================] - 21s 503ms/step - loss: 3.7740 - val_loss: 3.5460
Epoch 7/30
41/41 [==============================] - 19s 463ms/step - loss: 3.7897 - val_loss: 3.5460
Epoch 8/30
41/41 [==============================] - 19s 464ms/step - loss: 3.8054 - val_loss: 3.5460
Epoch 9/30
41/41 [==============================] - 19s 464ms/step - loss: 3.7425 - val_loss: 3.5460
Epoch 10/30
41/41 [==============================] - 21s 501ms/step - loss: 3.8054 - val_lo

In [0]:
# re-initialize our testing data generator, this time for evaluating
testGen = csv_image_generator(TEST_PATH, BS, lb,
    mode="eval", aug=None)

In [0]:

# make predictions on the testing images, finding the index of the
# label with the corresponding largest predicted probability
predIdxs = model.predict_generator(testGen,
    steps=(NUM_TEST_IMAGES // BS) + 1)
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print("[INFO] evaluating network...")
print(classification_report(testLabels.argmax(axis=1), predIdxs,
    target_names=lb.classes_))

In [0]:
imgpath = TEST_PATH
imgs = [x for x in os.listdir(imgpath) if x.split('.')[-1] == 'png']
print(len(imgs))
shuffle(imgs)
cnt = 0
labels = []
images = []
for fn in imgs:
    oimg = image.load_img(os.path.join(imgpath,fn), target_size=(224,224))
    oimg = image.img_to_array(oimg)
    num = int(fn.split('.')[0].split('-')[1])
    label = int(df.loc[num-1]["'src-data'"])
    images.append(oimg)
    labels.append(label)
labels = lb.transform(np.array(labels))
images = np.array(images)



250


In [0]:
predIdxs = model.predict(images)
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print("[INFO] evaluating network...")
print(classification_report(labels.argmax(axis=1), predIdxs))

[INFO] evaluating network...
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       250
           1       0.00      0.00      0.00         0

   micro avg       0.00      0.00      0.00       250
   macro avg       0.00      0.00      0.00       250
weighted avg       0.00      0.00      0.00       250



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
# plot the training loss and accuracy
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
# plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
# plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

In [0]:
predIdxs

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1])